In [57]:
import pandas as pd 
from sklearn.pipeline import Pipeline
import plotly.express as px
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np
from datetime import datetime, timedelta

pd.set_option("display.max_columns", 100)

In [58]:
import redis 
import pickle
redis_client = redis.Redis(host='localhost', port=6360)